In [ ]:
# Import libraries and set up directories
from pathlib import Path
import numpy as np
import pandas as pd
import json
import pickle
from sklearn.model_selection import train_test_split

NOTEBOOK_DIR = Path.cwd()
OUTPUT_DIR = NOTEBOOK_DIR / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

print("Current notebook directory:", NOTEBOOK_DIR)
print("Local outputs directory:", OUTPUT_DIR)

Current notebook directory: C:\Users\kvand\PycharmProjects\ARC\arc1\cnn_parallel_branch
Local outputs directory: C:\Users\kvand\PycharmProjects\ARC\arc1\cnn_parallel_branch\outputs


In [ ]:
# Set paths for data and features
DATA_DIR = NOTEBOOK_DIR.parents[1] / "data" / "arc1"
TRAIN_DIR = DATA_DIR / "training"
EVAL_DIR  = DATA_DIR / "evaluation"

FEATURES_DIR = NOTEBOOK_DIR.parents[1] / "arc1" / "top_feats_as_sigs" / "outputs"

print("Data path:", DATA_DIR)
print("Feature path:", FEATURES_DIR)

Data path: C:\Users\kvand\PycharmProjects\ARC\data\arc1
Feature path: C:\Users\kvand\PycharmProjects\ARC\arc1\top_feats_as_sigs\outputs


In [ ]:
# Load training and evaluation data
def load_directory(path):
    tasks = {}
    for file in path.glob("*.json"):
        with open(file, "r") as f:
            tasks[file.stem] = json.load(f)
    return tasks

training_raw = load_directory(TRAIN_DIR)
evaluation_raw = load_directory(EVAL_DIR)

print("Loaded:", len(training_raw), "training tasks")
print("Loaded:", len(evaluation_raw), "evaluation tasks")

Loaded: 400 training tasks
Loaded: 400 evaluation tasks


In [ ]:
# Convert task data to numpy arrays
def as_np(grid):
    return np.array(grid, dtype=int)

def convert_task(raw_task):
    return {
        "train_inputs":  [as_np(p["input"]) for p in raw_task["train"]],
        "train_outputs": [as_np(p["output"]) for p in raw_task["train"]],
        "test_inputs":   [as_np(p["input"]) for p in raw_task["test"]],
        "test_outputs":  [as_np(p["output"]) for p in raw_task["test"]],
    }

training = {tid: convert_task(t) for tid, t in training_raw.items()}
evaluation = {tid: convert_task(t) for tid, t in evaluation_raw.items()}

In [ ]:
# Load topological features
df_features = pd.read_pickle(FEATURES_DIR / "features_training.pkl")
print("Loaded feature table:", df_features.shape)

Loaded feature table: (60056, 21)


In [ ]:
# Aggregate features to grid level
group_cols = ["task_id", "grid_role", "grid_index"]
numeric_cols = df_features.select_dtypes(include=[int, float]).columns
numeric_cols = [c for c in numeric_cols if c not in group_cols]

df_grid = (
    df_features
    .groupby(group_cols)[numeric_cols]
    .mean()
    .reset_index()
)

df_train_grids = df_grid[df_grid["grid_role"].isin(["train_input", "train_output"])]
df_train_grids = df_train_grids.reset_index(drop=True)

print("Grid-level descriptors:", df_train_grids.shape)

Grid-level descriptors: (2604, 21)


In [ ]:
# Pad grids to 30x30
def pad_grid(grid, size=30):
    arr = np.array(grid, dtype=int)
    H, W = arr.shape
    out = -1 * np.ones((size, size), dtype=int)
    out[:H, :W] = arr[:size, :size]
    return out

In [ ]:
# Augment data by permuting colors
def permute_colors(grid):
    colors = np.arange(10)
    perm = np.random.permutation(colors)
    out = grid.copy()
    for old, new in zip(colors, perm):
        out[out == old] = new
    return out

def augment_grid(grid):
    return permute_colors(grid)

In [ ]:
# Prepare input data with images and features
X_img_list = []
X_feat_list = []
y_list = []

for _, row in df_train_grids.iterrows():

    tid  = row["task_id"]
    role = row["grid_role"]
    idx  = int(row["grid_index"])

    if role == "train_input":
        grid = training[tid]["train_inputs"][idx]
    else:
        grid = training[tid]["train_outputs"][idx]

    padded = pad_grid(grid)
    #padded = augment_grid(padded)
    X_img_list.append(padded)

    topo_vec = row[numeric_cols].to_numpy(float)
    X_feat_list.append(topo_vec)

    out_grid = training[tid]["train_outputs"][idx]
    y_list.append(int(np.array(out_grid).sum()) % 10)

X_img = np.stack(X_img_list)[..., None]
X_feat = np.stack(X_feat_list)
y = np.array(y_list)

print("Image tensor:", X_img.shape)
print("Topo tensor:", X_feat.shape)
print("Labels:", y.shape)

Image tensor: (2604, 30, 30, 1)
Topo tensor: (2604, 18)
Labels: (2604,)


In [ ]:
# Split data into train and validation
X_img_train, X_img_val, X_feat_train, X_feat_val, y_train, y_val = train_test_split(
    X_img, X_feat, y, test_size=0.15, random_state=0, stratify=y
)

print("Train size:", len(y_train))
print("Val size:", len(y_val))

Train size: 2213
Val size: 391


In [ ]:
# Define hybrid CNN-topo model
import tensorflow as tf
from tensorflow.keras import layers, models, Input, regularizers

L2 = regularizers.l2(1e-5)

def conv_block(x, f):
    x = layers.Conv2D(f, 3, activation="relu", padding="same", kernel_regularizer=L2)(x)
    x = layers.Conv2D(f, 3, activation="relu", padding="same", kernel_regularizer=L2)(x)
    x = layers.BatchNormalization()(x)
    return x

img_input = Input(shape=(30, 30, 1))

# Encoder
c1 = conv_block(img_input, 32)
p1 = layers.MaxPooling2D(2)(c1)

c2 = conv_block(p1, 64)
p2 = layers.MaxPooling2D(2)(c2)

# Bottleneck
b = conv_block(p2, 128)

# Decoder (aligned center cropping)
u2 = layers.UpSampling2D(2)(b)
c2c = layers.CenterCrop(14, 14)(c2)
u2 = layers.concatenate([u2, c2c])
c3 = conv_block(u2, 64)

u1 = layers.UpSampling2D(2)(c3)
c1c = layers.CenterCrop(28, 28)(c1)
u1 = layers.concatenate([u1, c1c])
c4 = conv_block(u1, 32)

# CNN embedding
cnn_vec = layers.GlobalAveragePooling2D()(c4)
cnn_vec = layers.Dense(64, activation="relu")(cnn_vec)
cnn_vec = layers.Dropout(0.3)(cnn_vec)

# Topo MLP branch
feat_input = Input(shape=(X_feat.shape[1],))
z = layers.Dense(128, activation="relu")(feat_input)
z = layers.Dropout(0.3)(z)
z = layers.Dense(64, activation="relu")(z)

# Merge
combined = layers.concatenate([cnn_vec, z])
final = layers.Dense(64, activation="relu")(combined)
output = layers.Dense(10, activation="softmax")(final)

model = models.Model(inputs=[img_input, feat_input], outputs=output)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 30, 30, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 30, 30,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 30, 30,    │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 30, 30,    │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 15, 15,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 15, 15,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 15, 15,    │     36,928 │ conv2d_2[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 7, 7, 64)  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 7, 7, 128) │     73,856 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 7, 7, 128) │    147,584 │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 128) │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 14, 14,    │          0 │ batch_normalizat… │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ center_crop         │ (None, 14, 14,    │          0 │ batch_normalizat… │
│ (CenterCrop)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 14, 14,    │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 192)              │            │ center_crop[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 14, 14,    │    110,656 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 14, 14,    │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 493,930 (1.88 MB)

 Trainable params: 493,290 (1.88 MB)

 Non-trainable params: 640 (2.50 KB)

In [ ]:
# Train the model
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=8,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=4,
        min_lr=1e-5
    )
]

history = model.fit(
    [X_img_train, X_feat_train],
    y_train,
    validation_data=([X_img_val, X_feat_val], y_val),
    epochs=80,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/24
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.1152 - loss: 3.2265 - val_accuracy: 0.1355 - val_loss: 2.3831 - learning_rate: 0.0010
Epoch 2/24
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.1437 - loss: 2.5276 - val_accuracy: 0.1279 - val_loss: 2.3677 - learning_rate: 0.0010
Epoch 3/24
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.1229 - loss: 2.4916 - val_accuracy: 0.1509 - val_loss: 2.3334 - learning_rate: 0.0010
Epoch 4/24
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.1324 - loss: 2.3996 - val_accuracy: 0.1662 - val_loss: 2.3149 - learning_rate: 0.0010
Epoch 5/24
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.1423 - loss: 2.3317 - val_accuracy: 0.1739 - val_loss: 2.2738 - learning_rate: 0.0010
Epoch 6/24
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.1577 - loss: 2.3200 - val_accuracy: 0.1355 - val_loss: 2.3145 - learning_rate: 0.0010
Epoch 7/24
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.1636 - loss: 2.3092 - val_acc

In [ ]:
# Evaluate on validation set
val_loss, val_acc = model.evaluate([X_img_val, X_feat_val], y_val, verbose=0)
print(f"Validation accuracy (hybrid): {val_acc:.4f}")

Validation accuracy (hybrid): 0.1944


In [ ]:
# Save model and history
model_path = OUTPUT_DIR / "hybrid_cnn_topo.keras"
model.save(model_path)

with open(OUTPUT_DIR / "hybrid_history.pkl", "wb") as f:
    pickle.dump(history.history, f)

print("Saved hybrid model to:", model_path)

Saved hybrid model to: C:\Users\kvand\PycharmProjects\ARC\arc1\cnn_parallel_branch\outputs\hybrid_cnn_topo.keras
